In [1]:
!pip install transformers==4.36.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [2]:
import multiprocessing
multiprocessing.cpu_count()

2

In [3]:
# Fetch Clean data
## If you are using the data by mounting the google drive, use the following :
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=False)

%cd /content/gdrive/MyDrive/FinalThesis/Final_Data/

Mounted at /content/gdrive
/content/gdrive/MyDrive/FinalThesis/Final_Data


In [4]:
import numpy as np
import pandas as pd
import time

In [5]:
import scipy
from transformers import RobertaTokenizer, RobertaForSequenceClassification

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
#Load Twitter Dataset
start = time.time()
tweet_clean = pd.read_csv("clean_data_20-01-2024.csv")
t = time.time() - start
print('Time taken to load Twitter Clean Data:',t)

Time taken to load Twitter Clean Data: 8.495644569396973


In [7]:
tweet_clean.dtypes

date_time     object
clean_text    object
dtype: object

In [8]:
tweet_clean['date_time'] = pd.to_datetime(tweet_clean['date_time'])
tweet_clean['clean_text'] = tweet_clean['clean_text'].astype(str)

In [9]:
tweet_clean.head()

,date_time,clean_text
0,2021-02-10 23:59:04,blue ridge bank shares halted nyse bitcoin atm...
1,2021-02-10 23:58:48,today thats thursday take friend btc wallet se...
2,2021-02-10 23:54:48,guys evening read article btc would like share
3,2021-02-10 23:54:33,btc big chance billion price bitcoin fx btc cr...
4,2021-02-10 23:54:06,network secured nodes today soon biggest bears...


In [10]:
tweet_clean.shape

(1048570, 2)

In [11]:
dataByMonth= tweet_clean[(tweet_clean['date_time'].dt.month == 2)]
dataByMonth = dataByMonth.dropna()

In [13]:
#ddf=tweet_clean.head(100)
ddf=dataByMonth

In [14]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
roberta_model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
labels = roberta_model.config.id2label.values()
from scipy import special
def getSenimentScore(clean_text):
  clean_data = roberta_tokenizer(clean_text, padding=True, truncation=True, return_tensors='pt')
  logits = roberta_model(**clean_data).logits
  scores = {k: v for k, v in zip(labels, scipy.special.softmax(logits.detach().numpy().squeeze()))}
  return scores

In [16]:
from tqdm import tqdm

In [17]:
start = time.time()
print(start)
tqdm.pandas(desc='Calculating Compound Score........')
ddf['compound_score'] = ddf['clean_text'].progress_apply(lambda x: getSenimentScore(x))
t = time.time() - start
print("Time taken to calculate RoBerta sentiment score :",t)

1714315093.078882


Calculating Compound Score........: 100%|██████████| 100/100 [00:23<00:00,  4.17it/s]

Time taken to calculate RoBerta sentiment score : 23.978980779647827



<ipython-input-17-ea4ba0c3ec73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['compound_score'] = ddf['clean_text'].progress_apply(lambda x: getSenimentScore(x))


In [18]:
ddf.head()

,date_time,clean_text,compound_score
0,2021-02-10 23:59:04,blue ridge bank shares halted nyse bitcoin atm...,"{'negative': 0.444158, 'neutral': 0.5439871, '..."
1,2021-02-10 23:58:48,today thats thursday take friend btc wallet se...,"{'negative': 0.020750564, 'neutral': 0.7528926..."
2,2021-02-10 23:54:48,guys evening read article btc would like share,"{'negative': 0.010917802, 'neutral': 0.8882656..."
3,2021-02-10 23:54:33,btc big chance billion price bitcoin fx btc cr...,"{'negative': 0.011846485, 'neutral': 0.4474995..."
4,2021-02-10 23:54:06,network secured nodes today soon biggest bears...,"{'negative': 0.03188404, 'neutral': 0.5191256,..."


In [20]:
today_date = time.strftime("%d-%m-%Y")
excel_filename = "roberta_compound_score_"+today_date+".csv"
print(excel_filename)
ddf.to_csv(excel_filename, index=False, header=True)

roberta_compound_score_28-04-2024.csv
